# QMCPy for Logistic Regression
This notebook will give examples of how to use QMCPy for Logistic Regression

In [1]:
from qmcpy import *
from numpy import *
from qmcpy.integrand.LR import LR

Say we have some data and we want to use logistic regression for a dataset regarding admission to graduate school. The likelihood function for logistic regression, $f:\mathbb{R}^{d+1} \to [0,\infty)$ takes the form
\begin{align*}
f(\boldsymbol{x}) & = \prod_{i = 1}^m 
\left(\frac{\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij} \right)}
{1+\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij}\right)}\right)^{t_i}
\left(1-\frac{\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij} \right)}{1+\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij}\right)}\right)^{1-t_i} \\  %equations need punctuation
& = \prod_{i = 1}^m 
\left(\frac{\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij} \right)}
{1+\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij}\right)}\right)^{t_i}
\left(\frac{1}{1+\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij}\right)}\right)^{1-t_i} \\
& = \prod_{i = 1}^m 
\left(\frac{\left[\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij} \right) \right]^{t_i} }
{1+\exp\left(x_0 + \sum_{j = 1}^d x_j s_{ij}\right)}\right)
\end{align*}

Function $f$ is defined under two parameters. Let a $m \times d$ matrix $\boldsymbol{S}$ and vector $\boldsymbol{t}$ be given. Also, let $\boldsymbol{S} = (s_{ij})_{i, j = 1}^{m,d}$ and $\boldsymbol{t} = (t_1, t_2, t_3, ..., t_m)^T \in \{0,1\}^m$. Matrix $\boldsymbol{S}$ contains the values of the predictors, and vector $\boldsymbol{x}$ stores values of binary responses.

The purpose of using logistic regression is to find a way to predict an event in a binary setting. For example, what are the chances a coin will land tails, what are the chances this person is infected with covid-19,  or what are the chances it will precipitate that day. In this case, we want to find logistic regression model, so that we could determine the chances of getting admitted using student's ranking, gre score, and gpa.

Below we have both $\boldsymbol{S}$ and $\boldsymbol{t}$. You can change "n" to be any integer between 0 and 400.

In [2]:
n = 10
data = genfromtxt('binary.csv', dtype=float, delimiter=',', skip_header = True)
s = data[:n, 1:]
t = data[:n, 0]

We have our $\boldsymbol{S}$ and $\boldsymbol{t}$, now what we want to do is define our dimensions. For this to work we must have the number of rows in S + 1 for the dimensions to work according to function $f$. We will add 1 dimension to get the wanted results. 

In [3]:
no,dim_s = s.shape
r = dim_s +1
dim = r+1

Since we have our dimensions such that dim = d, so that $\boldsymbol{x} = (x_0, x_1, x_2, ..., x_d)$. What we want is to estimate the integration of $f$ with respect to a normal prior. Which is,

\begin{align*}
\int_{\mathbb{R}^{d+1}} f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}
\end{align*}

where

\begin{align*}
\pi(\boldsymbol{x}) = \frac{\exp(-\frac{1}{2}(\boldsymbol{x} - \boldsymbol{\nu})^T \boldsymbol{\mathsf{\Sigma}}^{-1}(\boldsymbol{x} - \boldsymbol{\nu}))}{\sqrt{(2 \pi)^{d+1}|\mathbf{\Sigma}|}}
\end{align*}

such that the prior mean is $\boldsymbol{\nu}$ and prior covariance is $\mathsf{\Sigma}$. 

So that we can compute an array of $f(\boldsymbol{x}_1), f(\boldsymbol{x}_2), f(\boldsymbol{x}_3), ... f(\boldsymbol{x}_n)$ while $\boldsymbol{x}_i \sim \pi$, we need to calculate,

\begin{align*}
f(\boldsymbol{x}_k) = \prod_{i = 1}^m 
\left(\frac{\left[\exp\left(x_{k0} + \sum_{j = 1}^d x_{kj} s_{ij} \right)\right] ^{t_i} }
{1+\exp\left(x_{k0} + \sum_{j = 1}^d x_{kj} s_{ij}\right)} \right), k = 1, 2, 3, ..., n.
\end{align*}.

Now, let $r+1 \in \mathbb{N}$ such that $0 \leq r \leq d + 1$ and is the number of integrations we want to find,

\begin{align*}
\int_{\mathbb{R}^{d+1}} f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}, \int_{\mathbb{R}^{d+1}} x_1 f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}, \int_{\mathbb{R}^{d+1}} x_2 f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}, \int_{\mathbb{R}^{d+1}} x_3 f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}, ..., \int_{\mathbb{R}^{d+1}} x_{r-1}f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}.
\end{align*}

We want to calculate these integrations because they are related to the posterior means. In Bayesian inference, the posterior mean of $x_i$ while $0 \leq i \leq d$ is

\begin{align*}
\frac{\int_{\mathbb{R}^{d+1}} x_i  f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}}{\int_{\mathbb{R}^{d+1}}  f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}}
\end{align*}.

The first integral, $\int_{\mathbb{R}^{d+1}} f(\boldsymbol{x}) \pi(\boldsymbol{x})\, d \boldsymbol{x}$, is the denominator, while the other integrals are numerators. Now we calculate the integrations, below.

In [4]:
lr = LR(Sobol(dim_s+1,seed=8), s_matrix = s, t = t, r = r, prior_variance=[1,1e-4,1,1])
if r==0: raise Exception('require r>0')
qmcclt = CubQMCCLT(lr,
    abs_tol = 0,
    rel_tol = .25,
    n_init = 256,
    n_max = 2 ** 30,
    inflate = 1.2,
    alpha = 0.01,
    replications = 16,
    error_fun = lambda sv,abs_tol,rel_tol: maximum(abs_tol,abs(sv)*rel_tol),
    bound_fun = lambda phvl, phvh: (
        minimum.reduce([phvl[1:dim]/phvl[0],phvl[1:dim]/phvh[0],phvh[1:dim]/phvl[0],phvh[1:dim]/phvh[0]]),
        maximum.reduce([phvl[1:dim]/phvl[0],phvl[1:dim]/phvh[0],phvh[1:dim]/phvl[0],phvh[1:dim]/phvh[0]]),
        sign(phvl[0])!=sign(phvh[0])),
    dependency = lambda flags_comb: hstack((flags_comb.any(),flags_comb)))
s,data = qmcclt.integrate()
print(s)
print(data)

[-0.16430858  0.00694682 -0.65995462 -0.4110202 ]
MeanVarDataRep (AccumulateData Object)
    solution        [-0.164  0.007 -0.66  -0.411]
    indv_error_bound [5.689e-06 5.075e-06 1.399e-07 2.111e-05 1.052e-05]
    ci_low          [ 1.883e-04 -3.829e-05  1.248e-06 -1.547e-04 -9.313e-05]
    ci_high         [ 1.997e-04 -2.814e-05  1.528e-06 -1.125e-04 -7.209e-05]
    ci_comb_low     [-0.203  0.006 -0.821 -0.494]
    ci_comb_high    [-0.141  0.008 -0.563 -0.361]
    solution_comb   [-0.164  0.007 -0.66  -0.411]
    flags_comb      [False False False False]
    flags_indv      [False False False False False]
    n_total         2^(16)
    n               [4096. 4096.  512.  512.  512.]
    replications    2^(4)
    time_integrate  0.597
CubQMCCLT (StoppingCriterion Object)
    inflate         1.200
    alpha           0.010
    abs_tol         0
    rel_tol         2^(-2)
    n_init          2^(8)
    n_max           2^(30)
LR (Integrand Object)
Gaussian (TrueMeasure Object)
    mean    

So our values for our integrations are outputted in our solution. These values are actually our coefficients. So, let two vectors $\boldsymbol{c}$ and $\boldsymbol{x}$, such that $\boldsymbol{c} = (c_0, c_1, c_2, c_3, ..., c_{r-1})\in \mathbb{R}^r$ and $\boldsymbol{x} = (x_1, x_2, x_3, ..., x_{r-1}) \in \mathbb{R}^{r-1}$. Let $g: \mathbb{R}^{r-1} \to (0, 1)$ be a function. To map our function $g$ we have,

\begin{align*}
g(\boldsymbol{x}) = \frac{\exp(c_0 + c_1 x_1 + c_2 x_2 + c_3 x_3 + ... + c_{r-1} x_{r-1})}{1 + \exp(c_0 + c_1 x_1 + c_2 x_2 + c_3 x_3 + ... + c_{r-1} x_{r-1})}
\end{align*}

So, we have our logistic regression function to help predict things in a binary setting.


Now what we want to ask is how accurate these functions are. Like in linear regression there is a way to use error bound to see how accurate the function is. We have our lower and higher bounds outputted in the data so like we did for function $g$ we have, "ci_comb_low" which is the lower bound and "ci_comb_high" as our higher bound. For this example let $g_l: \mathbb{R}^{r-1} \to (0,1)$ be the lower bound function and $g_h: \mathbb{R}^{r-1} \to (0,1)$ be the higher bound function. Let two vectors exist such that $\boldsymbol{l} = (l_0, l_1, l_2, l_3, ..., l_{r-1})\in \mathbb{R}^r$ and $\boldsymbol{h} = (h_0, h_1, h_2, h_3, ..., h_{r-1})\in \mathbb{R}^r$. To map functions $g_l$ and $g_h$ we have,

\begin{align*}
g_l(\boldsymbol{x}) = \frac{\exp(l_0 + l_1 x_1 + l_2 x_2 + l_3 x_3 + ... + l_{r-1} x_{r-1})}{1 + \exp(l_0 + l_1 x_1 + l_2 x_2 + l_3 x_3 + ... + l_{r-1} x_{r-1})}
\end{align*}

and

\begin{align*}
g_h(\boldsymbol{x}) = \frac{\exp(h_0 + h_1 x_1 + h_2 x_2 + h_3 x_3 + ... + h_{r-1} x_{r-1})}{1 + \exp(h_0 + h_1 x_1 + h_2 x_2 + h_3 x_3 + ... + h_{r-1} x_{r-1})}
\end{align*}.

Thus, we have our error bounds for our logistic regression model